## Suman Thapaliya
## Visualization

# Introduction
In this Jupyter Notebook, we will go through pre-proceassing and visualization of Railway Accident dataset of European Countries from 2004 to 2015. The dataset is taken from Eurostat. It is openly avialble. Dataset explains how many people got injured or killed in an accident.
- Data Available here: https://ec.europa.eu/eurostat/de/web/main/data/database

## Dataset
Dataset has 4080 railway accident reported. It has 13 features. Some of the features and abbreviations can be interpreted as 
1. freq = Frequency represents how often the reports are collected. In our case, It is yearly.
2. Unit = Number
3. Accident = What kind of accident did happened? In our case, they are:
    - COLLIS = Collisions of trains, including collisions with obstacles within the clearance gauge
    - DERAIL = Derailments of trains
    - LVLCROS = Level crossing accidents
    - OTH = others
    - RSTK_FIRE = Fires in rolling stock
    - RSTK_MOT = Accidents to persons caused by rolling stock in motion
    - UNK = Unknown
4. victim = Did the Person got injury or died? They are represented as
    - INJ = Person was injured
    - KIL = Person was dead
5. pers_cat = Category of person who got into Accident categorized as 
    - EMP = Employee
    - OTH = Other
    - PAS = Passanger
    - UNK = Unknown
6. geo\TIME_PERIOD = Location of incident happened. In our case, it's a European Countries.
7. Year = Accident took place in between 2004 to 2015.





   

## Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import osmnx as ox
import geopandas as gpd

## Read TSV file
Dataset was in TSV foramt. TSV refers to Tab-Separated Values which is similar to CSV file, but seperated by tabs. 

In [2]:
df = pd.read_csv('railway_accident.tsv', sep='\t')
df.head(5)

freq,unit,accident,victim,pers_cat,geo\TIME_PERIOD 2004  2005  2006  2007   \
0                             A,NR,COLLIS,INJ,EMP,AT    3     6     2     0    
1                             A,NR,COLLIS,INJ,EMP,BE    4     3     0     4    
2                             A,NR,COLLIS,INJ,EMP,BG    :     :     0     2    
3                             A,NR,COLLIS,INJ,EMP,CH    :     :     :     :    
4                             A,NR,COLLIS,INJ,EMP,CY   : z   : z   : z   : z   

  2008  2009  2010  2011  2012  2013  2014  2015   
0    0     1     1     0     0     5     0     0   
1    4     0     2     0     1     0     0     0   
2    0     0     0     0     3     0     0     0   
3    0     0     0     1     1     0     0     2   
4   : z   : z   : z   : z   : z   : z   : z   : z

In [3]:
df.shape

(4080, 13)

# Pre-Processing
As you can see that one column named 'freq,unit,accident,victim,pers_cat,geo\TIME_PERIOD' has multiple information so we need to split it in order to get better insides while performing visualization.

In [4]:
new_columns = df['freq,unit,accident,victim,pers_cat,geo\\TIME_PERIOD'].str.split(',', expand=True)
new_columns.columns = ['freq', 'unit', 'accident', 'victim', 'pers_cat', 'geo_TIME_PERIOD']
df = pd.concat([new_columns, df.iloc[:, 1:]], axis=1)
df.head(3)

freq unit accident victim pers_cat geo_TIME_PERIOD 2004  2005  2006  2007   \
0    A   NR   COLLIS    INJ      EMP              AT    3     6     2     0    
1    A   NR   COLLIS    INJ      EMP              BE    4     3     0     4    
2    A   NR   COLLIS    INJ      EMP              BG    :     :     0     2    

  2008  2009  2010  2011  2012  2013  2014  2015   
0    0     1     1     0     0     5     0     0   
1    4     0     2     0     1     0     0     0   
2    0     0     0     0     3     0     0     0

### Convert string to Numeric

Since, all the data types are strings. we can not performed various operation like aggregation or grouping. so, we converted the value of years columns from string to numeric. The value of year had 2 special charaters. They are:
- : = not available
- z = not applicable.
we converted these special charaters to NAN value along with 0.

In [5]:
df.dtypes

freq               object
unit               object
accident           object
victim             object
pers_cat           object
geo_TIME_PERIOD    object
2004               object
2005               object
2006               object
2007               object
2008               object
2009               object
2010               object
2011               object
2012               object
2013               object
2014               object
2015               object
dtype: object

In [6]:
columns_to_convert = ['2004 ', '2005 ', '2006 ', '2007 ', '2008 ', '2009 ', '2010 ', '2011 ', '2012 ', '2013 ', '2014 ', '2015 ']
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')
#df.dtypes

### Add another colums from different dataset
In our railway accident dataset, countries name are written in short form and we wanted full name of countries for easy reading later on visualization. so, we searched for another dataset which has short form of country name and their full name. we found main_data.csv which has it. Dataset is also available in Eurostat.
- Dataset Link: https://ec.europa.eu/eurostat/web/gisco/overview

In [7]:
df_1 = pd.read_csv('country_name.csv')
df_1.head(3)

CNTR_ID                                CNTR_NAME  \
0      ET  Federal Democratic Republic of Ethiopia   
1      EV                                      NaN   
2      EW                                      NaN   

                      NAME_ENGL NAME_FREN ISO3_CODE          SVRG_UN  \
0                      Ethiopia  Éthiopie       ETH  UN Member State   
1  Eurasian Patent Organization       NaN       NaN              NaN   
2                           NaN       NaN       NaN              NaN   

          CAPT EU_STAT EFTA_STAT CC_STAT  NAME_GERM  
0  Addis-Ababa       F         F       F  Äthiopien  
1          NaN       F         F       F        NaN  
2          NaN       F         F       F        NaN

### Merge Datasets
We merged the two dataset on left join as we do not want to loose information from railway accident dataset. 

In [8]:
merged_df = pd.merge(df, df_1[['CNTR_ID', 'NAME_ENGL']], left_on='geo_TIME_PERIOD', right_on='CNTR_ID', how='left')
merged_df.head(3)

freq unit accident victim pers_cat geo_TIME_PERIOD  2004   2005   2006   \
0    A   NR   COLLIS    INJ      EMP              AT    3.0    6.0    2.0   
1    A   NR   COLLIS    INJ      EMP              BE    4.0    3.0    0.0   
2    A   NR   COLLIS    INJ      EMP              BG    NaN    NaN    0.0   

   2007   2008   2009   2010   2011   2012   2013   2014   2015  CNTR_ID  \
0    0.0    0.0    1.0    1.0    0.0    0.0    5.0    0.0    0.0      AT   
1    4.0    4.0    0.0    2.0    0.0    1.0    0.0    0.0    0.0      BE   
2    2.0    0.0    0.0    0.0    0.0    3.0    0.0    0.0    0.0      BG   

  NAME_ENGL  
0   Austria  
1   Belgium  
2  Bulgaria

### 

### Drop Unnecessary columns and Rename
Dropping the unncessary columns and renaming the remaining columns with appropriate name.

In [9]:
merged_df.drop(columns=['CNTR_ID', 'geo_TIME_PERIOD'], inplace=True)
merged_df.rename(columns={
    'freq': 'Frequency',
    'unit': 'Unit',
    'accident': 'Accident',
    'victim': 'Victim',
    'pers_cat': 'Type of Person',
    'NAME_ENGL': 'Country'
}, inplace=True)
merged_df.head(3)

Frequency Unit Accident Victim Type of Person  2004   2005   2006   2007   \
0         A   NR   COLLIS    INJ            EMP    3.0    6.0    2.0    0.0   
1         A   NR   COLLIS    INJ            EMP    4.0    3.0    0.0    4.0   
2         A   NR   COLLIS    INJ            EMP    NaN    NaN    0.0    2.0   

   2008   2009   2010   2011   2012   2013   2014   2015    Country  
0    0.0    1.0    1.0    0.0    0.0    5.0    0.0    0.0   Austria  
1    4.0    0.0    2.0    0.0    1.0    0.0    0.0    0.0   Belgium  
2    0.0    0.0    0.0    0.0    3.0    0.0    0.0    0.0  Bulgaria

### Melt the Dataset
It is necessary to melt the dataset in long format in order to use different operation like filter and grouping. Also, we dropped those rows which has NAN value i.e. just zero.

In [10]:
df_melted = pd.melt(merged_df, id_vars=['Frequency', 'Unit', 'Accident', 'Victim', 'Type of Person', 'Country'],
                    var_name='Year', value_name='Value')
df_melted = df_melted.dropna(subset=['Value'])
df_melted.head(3)

Frequency Unit Accident Victim Type of Person  Country   Year  Value
0         A   NR   COLLIS    INJ            EMP  Austria  2004     3.0
1         A   NR   COLLIS    INJ            EMP  Belgium  2004     4.0
5         A   NR   COLLIS    INJ            EMP  Czechia  2004     5.0

### Check unique value of different columns to reduce redundancy
we checked unique value of  different Columns to see any kind of repetition in different words. we saw that there is unique value called TOTAL i.e. (INJ+KIL) in Victim Column which is manipulation of data which might affect in our visualization interpretation. so, we decided to drop those rows which has Victim as TOTAL to make dataset as correct as possible.

In [11]:
df_melted['Victim'].unique()

array(['INJ', 'KIL', 'TOTAL'], dtype=object)

In [12]:
filtered_df = df_melted[df_melted['Victim'] != 'TOTAL']

In [13]:
filtered_df['Accident'].unique()

array(['COLLIS', 'DERAIL', 'LVLCROS', 'OTH', 'RSTK_FIRE', 'RSTK_MOT',
       'TOTAL', 'UNK'], dtype=object)

In [14]:
filtered_df = filtered_df[filtered_df['Accident'] != 'TOTAL']

In [15]:
filtered_df['Type of Person'].unique()

array(['EMP', 'OTH', 'PAS', 'TOTAL', 'UNK'], dtype=object)

In [16]:
filtered_df = filtered_df[filtered_df['Type of Person'] != 'TOTAL']

### Removing Non-European Countries from dataset
we found that the dataset contains the inforamtion of Non-European countries too. so, we decided to remove it.

In [17]:
filtered_df['Country'].unique()

array(['Austria', 'Belgium', 'Czechia', 'Germany', 'Denmark', 'Estonia',
       'Greece', 'Spain', 'Finland', 'France', 'Croatia', 'Hungary',
       'Ireland', 'Italy', 'Liechtenstein', 'Lithuania', 'Luxembourg',
       'Latvia', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania',
       'Sweden', 'Slovenia', 'Slovakia', 'Türkiye', 'United Kingdom',
       'Bulgaria', 'Switzerland', 'North Macedonia', 'Montenegro'],
      dtype=object)

In [18]:
non_eu_countries = ['Switzerland', 'Liechtenstein', 'Montenegro', 
                    'North Macedonia', 'Norway', 'Türkiye']
df_filtered = filtered_df[~filtered_df['Country'].isin(non_eu_countries)]
df_filtered.head(5)

Frequency Unit Accident Victim Type of Person  Country   Year  Value
0         A   NR   COLLIS    INJ            EMP  Austria  2004     3.0
1         A   NR   COLLIS    INJ            EMP  Belgium  2004     4.0
5         A   NR   COLLIS    INJ            EMP  Czechia  2004     5.0
6         A   NR   COLLIS    INJ            EMP  Germany  2004    19.0
7         A   NR   COLLIS    INJ            EMP  Denmark  2004     0.0

### Save as CSV File
We saved the pre-processed file as final.csv to use in another notebook for visualization.

In [19]:
df_filtered.to_csv('final.csv', index=False)

# Refrences
- https://chatgpt.com/c/f64531a9-6b63-4b2a-b2cc-871caf01bb4f, Model = GPT-3.5
- https://github.com/plotly
- https://plotly.com/python/time-series/
- https://plotly.com/python/hover-text-and-formatting/